<a href="https://colab.research.google.com/github/zqq66/seq2seq-model/blob/main/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [151]:
from __future__ import unicode_literals, print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext.data import Field,BucketIterator, TabularDataset
from torch.nn.utils.rnn import pad_sequence
from nltk.translate.bleu_score import SmoothingFunction
from torchtext.datasets import Multi30k

import numpy as np
import pandas as pd 

import nltk
import os
import re
import random
import math
import time
import spacy
from google.colab import drive

drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [152]:
MAXLENGTH = 10
BATCHSIZE = 128


train_path = 'drive/My Drive/CMPUT499/seq2seq/data.csv'
test_path = 'drive/My Drive/CMPUT499/seq2seq/data_test.csv'
valid_path = 'drive/My Drive/CMPUT499/seq2seq/data_valid.csv'
SRC = Field(lower=True, 
            init_token="<sos>", 
            eos_token="<eos>",
            fix_length=MAXLENGTH,
            include_lengths=True)

TRG = Field(lower=True, 
            init_token="<sos>", 
            eos_token="<eos>",
            fix_length=MAXLENGTH)


train_data = TabularDataset(train_path, 
                      format="csv", 
                      skip_header=True,
                      fields=[("src", SRC), ("trg", TRG)])

valid_data = TabularDataset(valid_path, 
                      format="csv", 
                      skip_header=True,
                      fields=[("src", SRC), ("trg", TRG)])

test_data = TabularDataset(test_path, 
                      format="csv", 
                      skip_header=True,
                      fields=[("src", SRC), ("trg", TRG)])
print(len(train_data))

3


In [153]:
SRC.build_vocab(train_data)
TRG.build_vocab(train_data)
print(SRC.vocab.stoi)
print(TRG.vocab.stoi)

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
            (train_data, valid_data, test_data), 
              batch_size = BATCHSIZE,
              sort_within_batch = True,
              sort_key=lambda x: len(x.src),
              sort=True,
              device = device)



defaultdict(<function _default_unk_index at 0x7fea407358c8>, {'<unk>': 0, '<pad>': 1, '<sos>': 2, '<eos>': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7})
defaultdict(<function _default_unk_index at 0x7fea407358c8>, {'<unk>': 0, '<pad>': 1, '<sos>': 2, '<eos>': 3, 'b': 4, 'e': 5, 'f': 6, 'a': 7, 'c': 8})


In [154]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.RNN(emb_dim, hid_dim, n_layers)
        
        # self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_len):
        
        #src = [src len, batch size]
        
        embedded = self.embedding(src)
        
        
        #embedded = [src len, batch size, emb dim]
                
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len.cpu())
                
        packed_outputs, hidden = self.rnn(packed_embedded)
                                 
        #packed_outputs is a packed sequence containing all hidden states
        #hidden is now from the final non-padded element in the batch
            
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
        # print("outputs", outputs)
        # print("hidden", hidden)
        # print("hidden", hidden)
            
        # outputs= self.rnn(embedded)
        # print(outputs)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [155]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.RNN(emb_dim, hid_dim, n_layers)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        # self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.embedding(input)
        
        #embedded = [1, batch size, emb dim]
                
        output, hidden = self.rnn(embedded, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        # print(prediction)
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [156]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden = self.encoder(src, src_len)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden = self.decoder(input, hidden)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [157]:
# size of vocabulary
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)

ENC_EMB_DIM = 512
DEC_EMB_DIM = 512
# ENC_HID_DIM = 15
# DEC_HID_DIM = 15
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0
DEC_DROPOUT = 0

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS)

model = Seq2Seq(enc, dec, device).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [158]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src, src_len = batch.src
        trg = batch.trg
        # print("src.shape", src.shape)
        optimizer.zero_grad()
        
        output = model(src, src_len, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [159]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src, src_len = batch.src
            trg = batch.trg

            output = model(src, src_len, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item() # add loss iteratively from batch
        
    return epoch_loss / len(iterator)

In [160]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = MAXLENGTH):

    model.eval()
        
    tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    # print("token with eos, sos", tokens)
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    src_len = torch.LongTensor([len(src_indexes)]).to(device)
    # print("token encoded", src_indexes)
    
    src_indexes += [src_field.vocab.stoi[src_field.pad_token] for i in range(max_len-len(src_indexes))] 

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
    # print("token tensor", src_indexes)

    with torch.no_grad():
        hidden = model.encoder(src_tensor, src_len)
        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]
    
    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                
        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden)

        pred_token = output.argmax(1).item()
        
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
        trg_indexes.append(pred_token)
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:]

# model.load_state_dict(torch.load('tut1-model.pt'))

# src = ['a', 'b', 'c', 'd']
# translation = translate_sentence(src, SRC, TRG, model, device)

# print('predicted trg = {', translation,'}')

In [161]:
N_EPOCHS = 50
CLIP = 1

best_bleu = float('-inf')


for epoch in range(N_EPOCHS):
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    BLEU = 0
    exact_match = 0
    for i in range(1, len(train_data)):
      src = train_data[i].src
      # print(f'src = {src}')

      translation = translate_sentence(src, SRC, TRG, model, device)
      trg = train_data[i].trg
      # print(f'trg = {trg}')
      # print(f'translation = {translation}')

      BLEU += nltk.translate.bleu_score.sentence_bleu(trg, translation)

      if trg == translation:
        exact_match += 1

    BLEU /= len(train_data)
    exact_match /= len(train_data)
    print(f'| Train BLEU: {BLEU:.3f} | Exact Match Accuracy: {exact_match:.3f} |')

  
    if exact_match > best_bleu:
        best_bleu = exact_match
        torch.save(model.state_dict(), 'tut1-model.pt')
        print('model saved in', epoch)

    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


| Train BLEU: 0.598 | Exact Match Accuracy: 0.000 |
model saved in 0
	Train Loss: 2.256 | Train PPL:   9.545
	 Val. Loss: 2.180 |  Val. PPL:   8.846
| Train BLEU: 0.561 | Exact Match Accuracy: 0.000 |
	Train Loss: 2.145 | Train PPL:   8.541
	 Val. Loss: 2.121 |  Val. PPL:   8.336
| Train BLEU: 0.550 | Exact Match Accuracy: 0.000 |
	Train Loss: 1.995 | Train PPL:   7.355
	 Val. Loss: 2.264 |  Val. PPL:   9.625
| Train BLEU: 0.513 | Exact Match Accuracy: 0.000 |
	Train Loss: 1.802 | Train PPL:   6.063
	 Val. Loss: 2.297 |  Val. PPL:   9.943
| Train BLEU: 0.627 | Exact Match Accuracy: 0.333 |
model saved in 4
	Train Loss: 1.699 | Train PPL:   5.466
	 Val. Loss: 2.338 |  Val. PPL:  10.365
| Train BLEU: 0.649 | Exact Match Accuracy: 0.667 |
model saved in 5
	Train Loss: 1.564 | Train PPL:   4.780
	 Val. Loss: 2.381 |  Val. PPL:  10.811
| Train BLEU: 0.649 | Exact Match Accuracy: 0.667 |
	Train Loss: 1.368 | Train PPL:   3.929
	 Val. Loss: 2.430 |  Val. PPL:  11.363
| Train BLEU: 0.649 | Exa

In [162]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 2.418 | Test PPL:  11.220 |


In [163]:
BLEU = 0
exact_match = 0
print(len(train_data))
for i in range(0, len(train_data)):
  src = train_data[i].src
  # print(f'src = {src}')

  translation = translate_sentence(src, SRC, TRG, model, device)
  trg = train_data[i].trg
  print(f'trg = {trg}')
  print(f'translation = {translation}')

  BLEU += nltk.translate.bleu_score.sentence_bleu(trg, translation)
  
  if trg == translation:
    exact_match += 1

BLEU /= len(train_data)
exact_match /= len(train_data)
print(f'| Train BLEU: {BLEU:.3f} | Exact Match Accuracy: {exact_match:.3f} |')


BLEU = 0
exact_match = 0
for i in range(0, len(test_data)):
  src = test_data[i].src
  # print(f'src = {src}')

  translation = translate_sentence(src, SRC, TRG, model, device)
  trg = test_data[i].trg
  # print(f'trg = {trg}')
  print(f'translation = {translation}')

  BLEU += nltk.translate.bleu_score.sentence_bleu(trg, translation)

  if trg == translation:
    exact_match += 1

BLEU /= len(test_data)
exact_match /= len(test_data)
print(f'| Test BLEU: {BLEU:.3f} | Exact Match Accuracy: {exact_match:.3f} |')


3
trg = ['a', 'e', 'f', 'b', 'e', 'b']
translation = ['a', 'e', 'f', 'b']
trg = ['f', 'a']
translation = ['f', 'a']
trg = ['c', 'c', 'e', 'b', 'f']
translation = ['c', 'c', 'e', 'b', 'f']
| Train BLEU: 0.982 | Exact Match Accuracy: 0.667 |
translation = ['c', 'c', 'e', 'b', 'f']
| Test BLEU: 0.795 | Exact Match Accuracy: 0.000 |


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [164]:
src = ['a', 'b', 'c', 'd']
translation = translate_sentence(src, SRC, TRG, model, device)

print('predicted trg = {', translation,'}')

predicted trg = { ['c', 'c', 'e', 'b', 'f'] }
